In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

def wavelength_to_rgb(wavelength, gamma=0.8):
    wavelength = float(wavelength)
    if wavelength >= 380 and wavelength <= 440:
        attenuation = 0.3 + 0.7 * (wavelength - 380) / (440 - 380)
        R = ((-(wavelength - 440) / (440 - 380)) * attenuation) ** gamma
        G = 0.0
        B = (1.0 * attenuation) ** gamma
    elif wavelength >= 440 and wavelength <= 490:
        R = 0.0
        G = ((wavelength - 440) / (490 - 440)) ** gamma
        B = 1.0
    elif wavelength >= 490 and wavelength <= 510:
        R = 0.0
        G = 1.0
        B = (-(wavelength - 510) / (510 - 490)) ** gamma
    elif wavelength >= 510 and wavelength <= 580:
        R = ((wavelength - 510) / (580 - 510)) ** gamma
        G = 1.0
        B = 0.0
    elif wavelength >= 580 and wavelength <= 645:
        R = 1.0
        G = (-(wavelength - 645) / (645 - 580)) ** gamma
        B = 0.0
    elif wavelength >= 645 and wavelength <= 750:
        attenuation = 0.3 + 0.7 * (750 - wavelength) / (750 - 645)
        R = (1.0 * attenuation) ** gamma
        G = 0.0
        B = 0.0
    else:
        R = 0.0
        G = 0.0
        B = 0.0
    return (R, G, B)

def reflectance(n1, n2, n3, wavelength, thickness):
    k = 2 * np.pi / wavelength
    r12 = (n1 - n2) / (n1 + n2)
    r23 = (n2 - n3) / (n2 + n3)
    return np.abs(r12 + r23 * np.exp(2j * n2 * k * thickness))**2

# Create directory
if not os.path.exists('simulation'):
    os.makedirs('simulation')

# Define the refractive index for air and glass
n1 = 1  # air
n3 = 1.5  # glass

# Define the thickness range
thickness = np.linspace(30e-9, 160e-9, 1000)  # in meters

# Define the wavelength for red, green and blue light
wavelengths = [650e-9, 550e-9, 450e-9]  # in meters
colors = ['red', 'green', 'blue']

# Define the refractive index range
n2_range = np.linspace(1.5, 2.5, 101)  # Step of 0.01

for i, n2 in enumerate(n2_range):
    # Create subplots for reflectance and color
    fig, axes = plt.subplots(2, 1, figsize=(8, 10))
    
    # Initialize an array to store the RGB values
    rgb_values = np.zeros((len(thickness), 3))

    for wavelength, color in zip(wavelengths, colors):
        # Compute the reflectance
        R = reflectance(n1, n2, n3, wavelength, thickness)

        # Convert the wavelength to RGB
        rgb = np.array(wavelength_to_rgb(wavelength * 1e9))

        # Add the RGB values to the array, weighted by the reflectance
        rgb_values += R[:, np.newaxis] * rgb

        # Plot the reflectance
        axes[0].plot(thickness * 1e9, R, color=rgb, label=color)  # thickness in nm

    # Normalize the RGB values
    rgb_values /= rgb_values.max(axis=0)

    # Plot the color as a function of thickness
    for t, rgb in zip(thickness, rgb_values):
        axes[1].axvline(t * 1e9, color=rgb, linewidth=2)  # thickness in nm

    axes[0].set_title('Reflectance of ITO film for n2={:.2f}'.format(n2))
    axes[0].set_xlabel('ITO thickness (nm)')
    axes[0].set_ylabel('Reflectance')
    axes[0].legend()

    axes[1].set_title('Color of ITO film for n2={:.2f}'.format(n2))
    axes[1].set_xlabel('ITO thickness (nm)')
    axes[1].set_yticks([])
    
    plt.tight_layout()

    # Save the plot
    plt.savefig('simulation/reflectance_color_{:.2f}.png'.format(n2))
